# Dynamic Pricing for Urban Parking Lots

In [ ]:
!pip install pathway bokeh pandas

In [11]:
import pathway as pw
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
output_notebook()


In [12]:
# Model-1

# Load dataset
df = pd.read_csv('/content/dataset.csv')

df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

# Ensure datetime sorting
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values(by=['SystemCodeNumber', 'timestamp'])

# Initialize constants
BASE_PRICE = 10.0
ALPHA = 5.0

# Initialize a column to hold price
df['price'] = np.nan

# Set starting prices per lot
initial_prices = {lot_id: BASE_PRICE for lot_id in df['SystemCodeNumber'].unique()}

# Apply the model
for lot_id in df['SystemCodeNumber'].unique():
    lot_data = df[df['SystemCodeNumber'] == lot_id].copy()
    previous_price = BASE_PRICE
    prices = []

    for _, row in lot_data.iterrows():
        occupancy = row['Occupancy']
        capacity = row['Capacity']

        if capacity == 0:
            price = previous_price  # Avoid division by zero
        else:
            # Linear price adjustment
            price = previous_price + ALPHA * (occupancy / capacity)

        # Store updated price and set as new previous_price
        prices.append(price)
        previous_price = price

    # Update prices in main DataFrame
    df.loc[df['SystemCodeNumber'] == lot_id, 'price'] = prices

# Optional: Save the result
df.to_csv('baseline_linear_model_output.csv', index=False)

# Preview result
print(df[['timestamp', 'SystemCodeNumber', 'Occupancy', 'Capacity', 'price']].head(20))

             timestamp SystemCodeNumber  Occupancy  Capacity      price
0  2016-10-04 07:59:00      BHMBCCMKT01         61       577  10.528596
1  2016-10-04 08:25:00      BHMBCCMKT01         64       577  11.083189
2  2016-10-04 08:59:00      BHMBCCMKT01         80       577  11.776430
3  2016-10-04 09:32:00      BHMBCCMKT01        107       577  12.703640
4  2016-10-04 09:59:00      BHMBCCMKT01        150       577  14.003466
5  2016-10-04 10:26:00      BHMBCCMKT01        177       577  15.537262
6  2016-10-04 10:59:00      BHMBCCMKT01        219       577  17.435009
7  2016-10-04 11:25:00      BHMBCCMKT01        247       577  19.575390
8  2016-10-04 11:59:00      BHMBCCMKT01        259       577  21.819757
9  2016-10-04 12:29:00      BHMBCCMKT01        266       577  24.124783
10 2016-10-04 13:02:00      BHMBCCMKT01        269       577  26.455806
11 2016-10-04 13:29:00      BHMBCCMKT01        263       577  28.734835
12 2016-10-04 14:02:00      BHMBCCMKT01        238       577  30

In [47]:
output_notebook()

# Example simulated data
df = pd.DataFrame({
    'timestamp': pd.date_range(start='2025-01-01 08:00', periods=18, freq='30min'),
    'price': [10 + i * 0.5 for i in range(18)]
})

# Create a ColumnDataSource
source = ColumnDataSource(df)

# Create the plot
p = figure(x_axis_type='datetime', width=800, height=300, title="Price Over Time")
p.line(x='timestamp', y='price', source=source, line_width=2)

# Show the plot inside Colab
show(p)


In [48]:
# Model-2
# Schema for incoming data
class Schema(pw.Schema):
    timestamp: int  # Change type to int or float
    lot_id: str
    occupancy: int
    capacity: int
    queue_length: int
    traffic: float
    is_special_day: int
    vehicle_type: str

# Replay with real timing
stream = pw.demo.replay_csv_with_time(
    path="dataset.csv",
    schema=Schema,
    time_column="timestamp",
    unit="ms"
)

In [49]:
# Assign weight mapping
weights = {"car":1.0, "bike":0.5, "truck":1.5}

@pw.udf
def compute_demand(
    occupancy: int,
    capacity: int,
    queue_length: int,
    traffic: float,
    is_special_day: int,
    vehicle_type: str
) -> float:
    # Access Pathway column values directly
    vehicle_weight = weights.get(vehicle_type, 0.0) # Handle potential missing keys
    occupancy_rate = occupancy / (capacity + 1e-6) # Avoid division by zero


    α, β, γ, δ, ε, λ = 0.6, 0.4, 0.3, 0.2, 0.5, 1.0
    base = 10
    demand_raw = (
        α * occupancy_rate +
        β * queue_length -  # Using raw queue_length
        γ * traffic +      # Using raw traffic
        δ * is_special_day +
        ε * vehicle_weight
    )


    price = max(5, min(20, base * (1 + λ * demand_raw * 0.1))) # Added a factor for reasonable price range

    return price

# Ingest streaming and apply price computation
processed = (
    stream.select(
        stream.timestamp,
        stream.lot_id, # Using lot_id as per schema
        stream.occupancy,
        stream.capacity,
        price=compute_demand( # Apply the UDF and name the resulting column 'price'
            occupancy=stream.occupancy,
            capacity=stream.capacity,
            queue_length=stream.queue_length,
            traffic=stream.traffic,
            is_special_day=stream.is_special_day,
            vehicle_type=stream.vehicle_type
        )
    )
)


In [ ]:
from bokeh.io import push_notebook

# Bokeh Setup
output_notebook()
source = ColumnDataSource({"timestamp": [], "price": []})
p = figure(x_axis_type="datetime", width=800, height=300, title="Live Price Stream")
p.line(x="timestamp", y="price", source=source, line_width=2)
handle = show(p, notebook_handle=True)

# Pathway Stream + Demand Calculation as before...

# Subscribe
def on_update(key, row, time, is_add):
    if row is not None:
        # Access data from the row dictionary provided by subscribe
        ts = pd.to_datetime(row["timestamp"]) # Assuming timestamp is a key
        price = row["price"] # Assuming price is a key
        source.stream({"timestamp": [ts], "price": [price]}, rollover=200)
        push_notebook(handle=handle)

pw.io.subscribe(processed, on_update)

# Run Pathway in a background thread
import threading
threading.Thread(target=pw.run).start()

In [ ]:
source = ColumnDataSource({"timestamp": [], "price": []})

p = figure(x_axis_type="datetime", width=800, height=300, title="Live Price Stream")
p.line(x="timestamp", y="price", source=source, line_width=2)
r = show(p, notebook_handle=True)
